In [13]:
import numpy as np
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
#%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
plt.rcParams['figure.figsize'] = [15, 5]
from IPython import display
from ipywidgets import interact, widgets

ff


In [ ]:
dataframe = pd.read_csv('https://raw.githubusercontent.com/PrzemyslawSarnacki/AirQualityPrediction/master/data/first_data.csv')